
# 🧨 Stress Testing Workbench

_Date generated: 2025-09-03_

A self‑contained notebook to run **portfolio stress tests** with factor and idiosyncratic shocks, 
**correlation spikes**, and **liquidity/impact** overlays. Uses your CSVs if present, else synthetic data.

**What you get**
- Load portfolio, prices/returns, and optional factor exposures.
- Scenario library: historical (1987, GFC, COVID-19), macro shocks, bespoke factor & vol shocks.
- Liquidity layer: spread widening, volume drought → slippage & capacity hit.
- Risk: instantaneous **PnL**, **VaR/ES**, heatmaps, tornado charts, waterfall of worst scenarios.
- Batch mode: run suite & export `reports/stress_summary.csv`.


## 0) Parameters

In [ ]:

# Optional inputs (CSV). If missing, synthetic will be created.
PATH_RETURNS = "data/returns.csv"   # wide: date, T1, T2, ...
PATH_WEIGHTS = "data/weights.csv"   # cols: ticker, weight
PATH_FACTORS = "data/factors.csv"   # wide: date, MKT, RATE, CRUDE, USD, ... (optional)
PATH_LOADINGS = "data/loadings.csv" # asset factor loadings: ticker, factor, beta (optional)

# Defaults
N_ASSETS = 12
N_DAYS = 800
RISK_FREE = 0.01

# Liquidity / cost knobs (bps)
BASE_SPREAD_BPS = 5.0
CRISIS_SPREAD_MULT = 3.0
TURNOVER_PER_SCENARIO = 0.5     # fraction of book turned in stress (for slippage proxy)
IMPACT_BPS_PER_10VOL = 4.0      # extra bps per +10% vol spike

SEED = 7


## 1) Setup

In [ ]:

import os, math, warnings
from typing import Dict, List, Tuple
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")
pd.options.display.float_format = "{:,.6f}".format
rng = np.random.default_rng(SEED)

def drawdown_curve(nav: pd.Series) -> pd.Series:
    return nav / nav.cummax() - 1.0

def pct(x): return 100.0 * x

def ensure_dir(p): 
    os.makedirs(os.path.dirname(p), exist_ok=True)
    return p


## 2) Load Data (CSV or Synthetic)

In [ ]:

def load_returns(path=PATH_RETURNS, n_assets=N_ASSETS, n_days=N_DAYS):
    if os.path.exists(path):
        return pd.read_csv(path, parse_dates=["date"]).set_index("date").sort_index()
    dates = pd.bdate_range("2022-01-01", periods=n_days)
    mu = rng.normal(0.0004, 0.0002, n_assets)
    sd = rng.uniform(0.01, 0.025, n_assets)
    data = [rng.normal(mu[i], sd[i], len(dates)) for i in range(n_assets)]
    cols = [f"A{i:02d}" for i in range(n_assets)]
    return pd.DataFrame(np.array(data).T, index=dates, columns=cols)

def load_weights(path=PATH_WEIGHTS, tickers=None):
    if os.path.exists(path):
        df = pd.read_csv(path)
        w = df.set_index("ticker")["weight"]
        return w / w.abs().sum()
    # Equal-weight default long-only
    w = pd.Series(1.0, index=tickers) / len(tickers) # type: ignore
    return w

def load_factors(path=PATH_FACTORS, start=None, end=None):
    if os.path.exists(path):
        f = pd.read_csv(path, parse_dates=["date"]).set_index("date").sort_index()
        return f.loc[start:end]
    # Synthetic factors
    idx = pd.bdate_range(start, end)
    factors = pd.DataFrame({
        "MKT": rng.normal(0.0003, 0.01, len(idx)),
        "RATE": rng.normal(0.0, 0.005, len(idx)),
        "CRUDE": rng.normal(0.0, 0.012, len(idx)),
        "USD": rng.normal(0.0, 0.006, len(idx)),
        "VOL": np.abs(rng.normal(0.0, 0.01, len(idx)))
    }, index=idx)
    return factors

def load_loadings(path=PATH_LOADINGS, tickers=None, factors=None):
    if os.path.exists(path):
        df = pd.read_csv(path)
        piv = df.pivot(index="ticker", columns="factor", values="beta").reindex(tickers).fillna(0.0)
        return piv
    # Random loadings for synthetic
    betas = pd.DataFrame(rng.normal(0, 0.7, size=(len(tickers), len(factors.columns))),# type: ignore
                         index=tickers, columns=factors.columns)# type: ignore
    # VOL factor loading is non-negative proxy
    betas["VOL"] = np.abs(betas["VOL"])
    return betas

rets = load_returns()
weights = load_weights(tickers=rets.columns)
factors = load_factors(start=rets.index.min(), end=rets.index.max())
betas = load_loadings(tickers=rets.columns, factors=factors)

rets.head(), weights.head(), factors.head(), betas.head()


## 3) Baseline Portfolio Risk

In [ ]:

port_ret = (rets * weights.reindex(rets.columns)).sum(axis=1)
nav = (1 + port_ret).cumprod()
baseline = {
    "CAGR": nav.iloc[-1]**(252/len(nav)) - 1,
    "Vol": port_ret.std()*np.sqrt(252),
    "Sharpe": (port_ret.mean()/port_ret.std())*np.sqrt(252) if port_ret.std()>0 else np.nan,
    "MaxDD": drawdown_curve(nav).min()
}
pd.Series(baseline)


## 4) Scenario Library

In [ ]:

def shock_vector(**kwargs):
    # keys over factor names; values in return units for a 1-day instantaneous shock
    return pd.Series(kwargs).reindex(betas.columns).fillna(0.0)

SCENARIOS = {
    "1987_Crash": shock_vector(MKT=-0.20, VOL=+0.20, USD=+0.02),
    "GFC_Liquidity": shock_vector(MKT=-0.08, RATE=-0.02, CRUDE=-0.06, VOL=+0.10),
    "COVID_Selloff": shock_vector(MKT=-0.12, RATE=-0.01, CRUDE=-0.10, USD=+0.03, VOL=+0.15),
    "Inflation_Spike": shock_vector(MKT=-0.04, RATE=+0.03, CRUDE=+0.05, USD=+0.01, VOL=+0.05),
    "Energy_OilShock": shock_vector(CRUDE=+0.12, MKT=-0.03, RATE=+0.005, VOL=+0.03),
    "USD_Crush": shock_vector(USD=+0.05, MKT=-0.02),
    "Rates_Tantrum": shock_vector(RATE=+0.04, MKT=-0.05, VOL=+0.06),
}

# Bespoke grid generator (e.g., +/- X% on each factor)
def grid_scenarios(level=0.05):
    out = {}
    for f in betas.columns:
        out[f"+{f}_{level:.1%}"] = shock_vector(**{f: +level})
        out[f"-{f}_{level:.1%}"] = shock_vector(**{f: -level})
    return out

SCENARIOS.update(grid_scenarios(0.03))
len(SCENARIOS)


## 5) Liquidity & Cost Overlay

In [ ]:

def liquidity_cost(spread_bps=BASE_SPREAD_BPS, spread_mult=1.0, turnover=TURNOVER_PER_SCENARIO, vol_spike=0.0):
    spread = spread_bps * spread_mult
    slip_bps = spread * turnover
    impact_bps = IMPACT_BPS_PER_10VOL * (vol_spike/0.10)
    return (slip_bps + impact_bps) / 1e4  # to return units

def apply_scenario(shock_name, svec: pd.Series, weights: pd.Series, betas: pd.DataFrame):
    # Factor-driven instantaneous PnL: asset_ret = betas @ shock + idio (ignore idio here)
    asset_shock = betas.values @ svec.values
    asset_shock = pd.Series(asset_shock, index=betas.index)
    pnl_no_cost = float((asset_shock * weights).sum())
    # liquidity penalty: spread widening & vol spike assumption from 'VOL' factor
    vol_spike = max(0.0, float(svec.get("VOL", 0.0)))
    spread_mult = CRISIS_SPREAD_MULT if vol_spike>0 else 1.0
    cost = liquidity_cost(spread_mult=spread_mult, vol_spike=vol_spike)
    pnl_after_cost = pnl_no_cost - cost
    out = {
        "scenario": shock_name,
        "gross_pnl": pnl_no_cost,
        "liq_cost": -cost,
        "net_pnl": pnl_after_cost,
        "vol_spike": vol_spike,
        "spread_mult": spread_mult
    }
    return out, asset_shock

# Run all scenarios
rows = []
asset_impacts = {}
for name, svec in SCENARIOS.items():
    out, ashock = apply_scenario(name, svec, weights, betas)
    rows.append(out); asset_impacts[name] = ashock

stress_df = pd.DataFrame(rows).set_index("scenario").sort_values("net_pnl")
stress_df.head()


## 6) VaR / ES (Historical) & Scenario Overlay

In [ ]:

hist = (rets * weights.reindex(rets.columns)).sum(axis=1).dropna()
alpha = 0.99
VaR = -np.percentile(hist, (1-alpha)*100)
ES = -hist[hist <= -VaR].mean()

risk_table = pd.DataFrame({
    "Hist_VaR_99": [VaR],
    "Hist_ES_99": [ES],
    "Worst_Stress_Net": [stress_df["net_pnl"].min()],
    "Median_Stress_Net": [stress_df["net_pnl"].median()]
})
risk_table


## 7) Visuals

In [ ]:

# Tornado (sorted net pnl)
sorted_net = stress_df["net_pnl"].sort_values()
plt.figure(figsize=(10,4))
plt.barh(sorted_net.index, sorted_net.values)
plt.title("Scenario Net PnL (Tornado)")
plt.tight_layout(); plt.show()

# Heatmap of asset impacts for worst scenarios
worst_names = stress_df.nsmallest(6, "net_pnl").index.tolist()
heat = pd.DataFrame({k: asset_impacts[k] for k in worst_names})
plt.figure(figsize=(10,4))
plt.imshow(heat.values.T, aspect='auto')
plt.yticks(range(len(worst_names)), worst_names)
plt.xticks(range(len(heat.index)), heat.index, rotation=90)
plt.title("Asset Impact (Worst Scenarios)")
plt.colorbar(); plt.tight_layout(); plt.show()

# Waterfall for worst scenario
wname = worst_names[0]
gross = stress_df.loc[wname, "gross_pnl"]
cost = stress_df.loc[wname, "liq_cost"]
net = stress_df.loc[wname, "net_pnl"]
plt.figure(figsize=(8,3.2))
vals = [0, gross, gross+cost, net]  # start at 0 -> gross -> +cost -> net
labels = ["Start", "Gross", "Cost", "Net"]
plt.plot([0,1,2,3], vals, marker="o")
for i,(x,y) in enumerate(zip(range(4), vals)):
    plt.text(x, y, f"{y:.2%}", ha="center", va="bottom")
plt.xticks(range(4), labels)
plt.title(f"Waterfall — {wname}")
plt.tight_layout(); plt.show()


## 8) Export Summary

In [ ]:

summary = stress_df.copy()
summary["rank"] = summary["net_pnl"].rank(ascending=True, method="dense")
ensure_dir("reports/stress_summary.csv")
summary.to_csv("reports/stress_summary.csv")
"reports/stress_summary.csv"


## 9) Sensitivity Sweeps

In [ ]:

def sweep_factor(factor, lows=-0.10, highs=0.10, steps=21):
    xs = np.linspace(lows, highs, steps)
    results = []
    for x in xs:
        s = shock_vector(**{factor: x})
        out, _ = apply_scenario(f"{factor}@{x:.1%}", s, weights, betas)
        results.append((x, out["net_pnl"]))
    df = pd.DataFrame(results, columns=["shock","net_pnl"])
    return df

sv = sweep_factor("MKT")
plt.figure(figsize=(8,3))
plt.plot(pct(sv["shock"]), pct(sv["net_pnl"]))
plt.axhline(0, color="black", lw=1)
plt.xlabel("MKT shock (%)"); plt.ylabel("Net PnL (%)")
plt.title("Sensitivity — Market Shock")
plt.tight_layout(); plt.show()
